# Audio Chunking
### Airplane Landing Source Data
Last Modified August 6, 2019

In [52]:
from __future__ import print_function
import matplotlib
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
from IPython.display import SVG
import pandas as pd
import numpy as np
import os
import os.path
from os.path import isfile, join, listdir
import seaborn as sns
import fnmatch
import random
import sklearn
from sklearn import preprocessing
import crepe

import scipy.io.wavfile as wavfile
import pickle
np.set_printoptions(suppress=True)
import librosa
import resampy



ImportError: cannot import name 'listdir'

In [ ]:
#Define folder paths
cwd = "D:\\"

root = os.path.join(cwd,"aircraft-audio")
pip 
laxroot = os.path.join(cwd, "aircraft-audio","lax-naming-convention")
searoot = os.path.join(cwd, "aircraft-audio","sea-naming-convention")
os.chdir(laxroot)
os.getcwd()
roots = {laxroot,searoot}
print(root)

### Ingest, Resample, Sort and Slice Audio Data

In [ ]:
# Change from mp3 to wav file
from pydub import AudioSegment
for root in roots:
    for subdir, dirs, files in os.walk(root):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath.endswith(".mp3"):
                sound = AudioSegment.from_mp3(filepath)
                filepath1 = os.path.splitext(filepath)[0] + '.wav'
                sound.export(filepath1, format="wav")


In [ ]:
# change sample rate to 32000
resample_rate = 32000
for root in roots:
    for subdir, dirs, files in os.walk(root):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath.endswith(".wav"):
                fs, data = wavfile.read(filepath)
                y = resampy.resample(data, fs, resample_rate)
                wavfile.write(filepath, resample_rate, y)
            

In [ ]:
filename1 = "2019_07_21_17_07_47-LAX-A321-AAL369-DFW.wav"
fs, data = wavfile.read(filename1)
print(filename1)
plt.plot(data)
plt.show()

### Create lists of aircraft classes

In [ ]:
# Initialize vars, create lists of aircraft classes
laxentries = os.listdir(laxroot)
seaentries = os.listdir(searoot)
alllists = [laxentries,seaentries]
listnames = [laxroot,searoot]
Boeing700series = []
Embraer75series = []
OtherAircraft = []
Boeing700series_path = []
Embraer75series_path = []
OtherAircraft_path = []
iter = 0

os.chdir(root)
for lists in alllists:
    for file_name in lists:
        dir_name = os.path.join(listnames[iter]+os.sep+ file_name)
        print(dir_name)
        if file_name.endswith('.wav'):
            if fnmatch.fnmatch(file_name, '*B7*'):
                Boeing700series.append(file_name)
                Boeing700series_path.append(dir_name)
                
            elif fnmatch.fnmatch(file_name, '*E75*'):
                Embraer75series.append(file_name)
                Embraer75series_path.append(dir_name)

            else:
                OtherAircraft.append(file_name)
                OtherAircraft_path.append(dir_name)
                
    iter = iter+1         

### Create train, test split of flights

In [ ]:
#Define folder paths to write out seperated test train samples
cwd = "D:\\aircraft-audio\\"
Train_Boeing_dir = os.path.join(cwd, "Train","Train_Boeing")
Train_Embraer_dir = os.path.join(cwd, "Train","Train_Embraer")
Train_Other_dir = os.path.join(cwd, "Train","Train_Other")

Test_Boeing_dir = os.path.join(cwd, "Test","Test_Boeing")
Test_Embraer_dir = os.path.join(cwd, "Test","Test_Embraer")
Test_Other_dir = os.path.join(cwd, "Test","Test_Other")

os.chdir(root)
os.getcwd()

filteredlists_dir = [
Train_Boeing_dir,Train_Embraer_dir,Train_Other_dir,
Test_Boeing_dir,Test_Embraer_dir,Test_Other_dir
]


random.shuffle(Boeing700series)
random.shuffle(Embraer75series)
random.shuffle(OtherAircraft)

Boeinglength = len(Boeing700series)
Embraerlength = len(Embraer75series)
Otherlength = len(OtherAircraft)

Boeingsplit = int((Boeinglength/3)*2)
Embraersplit = int((Embraerlength/3)*2)
Othersplit = int((Otherlength/3)*2)

Train_Boeing = Boeing700series[:Boeingsplit]
Train_Embraer = Embraer75series[:Embraersplit]
Train_Other = OtherAircraft[:Othersplit]

Test_Boeing = Boeing700series[(Boeingsplit):]
Test_Embraer = Embraer75series[(Embraersplit):]
Test_Other = OtherAircraft[(Othersplit):]

In [ ]:
# review full paths for source
readpaths = [Boeing700series_path,
Embraer75series_path,
OtherAircraft_path]
len(readpaths)

In [60]:
root = "D:\\aircraft-audio"
os.chdir(root)
cwd = os.getcwd()

In [61]:
print(root)

D:\aircraft-audio


In [62]:
# read sample split files and write files new target test and train directories

def _write_audio_to_train_test(Class_paths, Name_list,Class, Split):
    iter = 0
    for file in Name_list:
        if iter < Split:
            write_dir_name = os.path.join(cwd+ os.sep+ "Train"+os.sep+"Train_"+Class+os.sep+file)
        else:
            write_dir_name = os.path.join(cwd+  os.sep+ "Test"+os.sep+"Test_"+Class+os.sep+file)
        fs, data = wavfile.read(Class_paths[iter])
        wavfile.write(write_dir_name, fs, data)
        iter = iter+1
            


In [63]:
_write_audio_to_train_test(Boeing700series_path, 
                           Boeing700series,
                           "Boeing", 
                           Boeingsplit )  

In [64]:
_write_audio_to_train_test(Embraer75series_path, 
                           Embraer75series,
                           "Embraer", 
                           Embraersplit )

In [65]:
_write_audio_to_train_test(OtherAircraft_path, 
                           OtherAircraft,
                           "Other", 
                           Othersplit )  

### Slice Audio to Desired Lengths

In [66]:
def _slice_audio_data(raw_audio_data, interval_len=1):
    accounted_fs = int(resample_rate * interval_len)
    num_slices = int(len(raw_audio_data) / accounted_fs)
    audio_data = raw_audio_data[: int(num_slices * accounted_fs)]  # Trims audio array
    sliced_audio = np.split(audio_data, num_slices)
    return np.array(sliced_audio)

In [67]:

def _slice_audio(source_dir, target_dir, slice_time):
    num = 0
    for subdir, dirs, files in os.walk(source_dir):
        os.chdir(source_dir)
        filepath = "" 

        for file in files:

                filepath = subdir + os.sep + file
                path_list = filepath.split(os.sep)
                path_adder = path_list[-2]
                if filepath.endswith(".wav"):
                    fs, data = wavfile.read(filepath)
                    sliced_audio = _slice_audio_data(data, 1)
                    sliced_audio = sliced_audio[0:slice_time,:]

                    target_dir_specific = os.path.join(target_dir + os.sep+ path_adder)
                    os.chdir(target_dir_specific)

                    for i in range(sliced_audio.shape[0]):
                        filename, file_extension = os.path.splitext(file)
                        new_fname = "{}-{}{}".format(filename,  i, file_extension)
                        if i > 0:
                            wavfile.write(new_fname, resample_rate, sliced_audio[i,:])



In [68]:
_slice_audio("D:\\aircraft-audio\\Train\\",
             "D:\\aircraft-audio\\Train_Slices", 
             60)

_slice_audio("D:\\aircraft-audio\\Test\\",
             "D:\\aircraft-audio\\Test_Slices", 
             60)

In [69]:
#Define folder paths
cwd = "D:\\"

Boeing_slices_train = os.path.join(cwd, "aircraft-audio","Train_Slices","Train_Boeing")
Boeing_slices_test =os.path.join(cwd, "aircraft-audio","Test_Slices","Test_Boeing")

Embraer_slices_train = os.path.join(cwd, "aircraft-audio","Train_Slices","Train_Embraer")
Embraer_slices_test =os.path.join(cwd, "aircraft-audio","Test_Slices","Test_Embraer")

Other_slices_train = os.path.join(cwd, "aircraft-audio","Train_Slices","Train_Other")
Other_slices_test =os.path.join(cwd, "aircraft-audio","Test_Slices","Test_Other")


In [71]:
listofdirs = [Boeing_slices_test,Boeing_slices_train,
              Embraer_slices_train,Embraer_slices_train,
              Other_slices_train,Other_slices_test]
Boeing_slices_test_matrix = []
Boeing_slices_train_matrix = []
Embraer_slices_train_matrix = []
Embraer_slices_test_matrix = []
Other_slices_train_matrix = []
Other_slices_test_matrix = []

In [89]:
def _create_numpy_arrays(each_dir, target_numpy):
    count = 0

    for subdir, dirs, files in os.walk(each_dir):
        for file in files:
            filepath = subdir + os.sep + file

            fs, raw_audio = wavfile.read(filepath)
            count = count +1
            target_numpy.append((raw_audio))
        print(count)   
    return(target_numpy)


In [90]:
Boeing_slices_train_matrix = _create_numpy_arrays(Boeing_slices_train,Boeing_slices_train_matrix)
Boeing_slices_test_matrix = _create_numpy_arrays(Boeing_slices_test,Boeing_slices_test_matrix)

Embraer_slices_train_matrix = _create_numpy_arrays(Embraer_slices_train,Embraer_slices_train_matrix)
Embraer_slices_test_matrix = _create_numpy_arrays(Embraer_slices_test,Embraer_slices_test_matrix)

Other_slices_train_matrix = _create_numpy_arrays(Other_slices_train,Other_slices_train_matrix)
Other_slices_test_matrix = _create_numpy_arrays(Other_slices_test,Other_slices_test_matrix)


147
121
86
68
58
43


In [91]:
len(Boeing_slices_train_matrix)

294

In [92]:
# Convert from nested list to array
def _convert_nested_list_to_array(each_dir, target_numpy, size):
    target_numpy = np.asarray(target_numpy)
    new_target_numpy = np.empty((32000),int)
    
    for row in range(1,size,1):
        newrow = target_numpy[row]
        new_target_numpy = np.vstack([new_target_numpy, newrow])
    
    new_target_numpy - np.array(new_target_numpy)
    
    return(new_target_numpy)


In [93]:
Boeing_slices_train_matrix = _convert_nested_list_to_array(Boeing_slices_train,Boeing_slices_train_matrix, 147)
Boeing_slices_test_matrix = _convert_nested_list_to_array(Boeing_slices_test,Boeing_slices_test_matrix, 121)

Embraer_slices_train_matrix = _convert_nested_list_to_array(Embraer_slices_train,Embraer_slices_train_matrix, 86)
Embraer_slices_test_matrix = _convert_nested_list_to_array(Embraer_slices_test,Embraer_slices_test_matrix, 68)

Other_slices_train_matrix = _convert_nested_list_to_array(Other_slices_train,Other_slices_train_matrix, 58)
Other_slices_test_matrix = _convert_nested_list_to_array(Other_slices_test,Other_slices_test_matrix, 43)


### Save Array of Chunked Audio Locally

In [94]:
os.chdir("D:\\aircraft-audio\\local-data")
print(os.getcwd())

np.savetxt('Boeing_slices_train_matrix.txt', Boeing_slices_train_matrix, delimiter=',')
np.savetxt('Boeing_slices_test_matrix.txt', Boeing_slices_test_matrix, delimiter=',')
         
np.savetxt('Embraer_slices_train_matrix.txt', Embraer_slices_train_matrix, delimiter=',')
np.savetxt('Embraer_slices_test_matrix.txt', Embraer_slices_test_matrix, delimiter=',')
         
np.savetxt('Other_slices_train_matrix.txt', Other_slices_train_matrix, delimiter=',')
np.savetxt('Other_slices_test_matrix.txt', Other_slices_test_matrix, delimiter=',')

D:\aircraft-audio\local-data


### Inspect Audio

In [95]:

Boeing_slices_train_matrix = Boeing_slices_train_matrix.astype(float)
Boeing_slices_test_matrix = Boeing_slices_test_matrix.astype(float)
Embraer_slices_train_matrix = Embraer_slices_train_matrix.astype(float)
Embraer_slices_test_matrix = Embraer_slices_test_matrix.astype(float)
Other_slices_train_matrix = Other_slices_train_matrix.astype(float)
Other_slices_test_matrix = Other_slices_test_matrix.astype(float)

In [ ]:
D_left_Boeing_slices_train_matrix = np.abs(librosa.core.ifgram(Boeing_slices_train_matrix[20,:], center=False))
D_left_Boeing_slices_train_matrix.shape

In [ ]:
import librosa.display
import matplotlib.pyplot as plt
librosa.display.specshow(librosa.amplitude_to_db(D_left_Boeing_slices_train_matrix[1,:,:], ref=np.max),
                          y_axis='log', x_axis='time')
plt.title('Power spectrogram')
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()

In [ ]:
new_sample = np.abs(librosa.core.ifgram(Boeing_slices_train_matrix[10,:], center=False))

In [ ]:
import librosa.display
import matplotlib.pyplot as plt
librosa.display.specshow(librosa.amplitude_to_db(new_sample[1,:,:], ref=np.max),
                          y_axis='log', x_axis='time')
plt.title('Power spectrogram')
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()

In [ ]:

C = np.abs(librosa.cqt(new_sample[1,2,:], sr=32000))
librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
                         sr=32000, x_axis='time', y_axis='cqt_note')
plt.colorbar(format='%+2.0f dB')
plt.title('Constant-Q power spectrum')
plt.tight_layout()